# Проект курса ML1 Анализ веб-документов

Во множестве прикладных задач возникает необходимость разбить веб-страницы на какие-то определенные группы, где в каждой группе страницы будут очень похожи по смыслу. Например, представим, что Вы владелец сервиса интернет рекламы. Вашим клиентам хочется, чтобы их услуги рекламировались не на каком-то определенном сайте, а на всех сайтах их тематики. То есть Вам нужна какая-то тематическая разметка сайтов в интернете по множеству тематик, и клиент сможет выбрать любую, какая ему больше нравится. Как же такую разметку сделать?

Самый простой способ - разметить людьми множество сайтов в интернете на множество тем, настроить на этой выборке многоклассовый классификатор и сделать предсказанием на всем оставшемся "интернете". Однако, в таком подходе возникает несколько проблем. Во-первых, непонятно откуда брать само множество тем, на которое размечать. Во-вторых, даже если множество тем зафиксировано, не очень понятно, откуда для каждой темы брать примеры, чтобы добавить их в обучающее множество.

В данном задании мы предлагаем Вам попробовать другое решение. Пусть тематику задают сами данные! Разделим наши веб-страницы на множество групп, например, просто по словам в веб-страницах. В такой группе буду как документы об одном и том же, так и "аномалии", которые имеют схожие слова, но не соответствуют документам основной тематики. Например, в такой группе может содержаться подмножество веб-страниц про "ремонт пластиковых окон" и аномалии вроде "пластиковые игрушки", "ремонт квартир" и так далее. Нам останется только выделить подмножество документов одной темы, то есть все документы, которые про "ремонт пластовых окон" и убрать все аномалии. Затем подмножество как-то проименуем, чтобы показать клиенту, но этим Вы уже займетесь, когда будете продавать Вашу систему :)

В задании Вам предлагается работать с 28026 веб-страницами, которые уже скачаны и лежат в архиве content.tar.gz. Эти страницы разбиты по группам, каждая группа около 100 страниц. Каждая группа соответствует какой-то определенной теме, которая Вам неизвестна. Обучающее множество состоит из 129 групп. В обучающих группах ручной разметкой было проставлено, соответствует ли данный документ теме группы (target = 1) или это аномалия (target = 0). Тестовое множество состоит из 180 групп. Вам необходимо проставить для них target. Важно отметить, что обучающие и тестовые группы не пересекаются. Гарантируется, что в каждой группе есть подмножество документов из ее темы.

Далее будем называть веб-страницы, которые соотвествуют теме группы, "настоящими", а которые не соотвествуют - "выбросами". 

Контент веб-страниц

Лежит в архиве content.tar.gz. В директориии 28026  веб-страниц, с которыми мы будем работать в конкурсе. Во всех файлах .dat первой строчкой указан урл веб-страницы, на случай, если Вы захотите скачать веб-страницу самостоятельно или использовать урл в качестве признаков. 

In [2]:
import codecs
import re
from time import sleep
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import Stemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [3]:
nltk.download('stopwords')
stopWords = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nastyboget/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def clean_page(id):
    path = 'content/{}.dat'.format(id)
    remove = ['script', 'style',  'meta', 'label', 'textarea', 'a']
    with codecs.open(path, 'r', 'utf-8') as f:
        content = BeautifulSoup(f, 'lxml')
        for script in content(remove):
            script.extract()
        # get text
        text = content.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = [chunk for chunk in chunks if chunk]
        # remove url
        text = ' '.join(text[1:])
        text = text.lower()
        text = re.sub('[^A-Za-zа-яёА-ЯЁ]+', ' ', text)
        text = re.sub('\s+', ' ', text)
        words = word_tokenize(text)
        wordsFiltered = []
        for w in words:
            if w not in stopWords:
                wordsFiltered.append(w)
        stemmer = Stemmer.Stemmer('russian')
        text = ' '.join(list(map(stemmer.stemWord, wordsFiltered)))
    return text.strip()

In [5]:
clean_page(1)

'м б аншин центр репродукц генетик фертимед г москв м б аншин центр репродукц генетик фертимед г москв kb названиедат конвертац размер kb типсодержан принцип гормональн анализ лечен бесплод показан интерпретац результат ошибк клиническ лекц м б аншин центр репродукц генетик фертимед г москварезюм стат систематизирова клиническ лабораторн дан свидетельств эндокрин расстройств приводя бесплод сопутств дан рекомендац рационализац минимизац числ лабораторн анализ этап обследован пациент бесплод ключев слов бесплод эндокринолог лабораторн анализвведен гормональн эндокрин нарушен могут самостоятельн причин бесплод фактор сопутств люб причин сил оценк гормональн норм патолог явля одн обязательн задач встающ врач обращен нем бесплодн супружеск пар богатств клиническ картин эндокрин нарушен позволя перв знакомств пациентк сформирова гипотез высок степен вероятн предполож отсутств налич тех ин отклонен гормональн статус очеред широк спектр лабораторн метод дает возможн объективн измер содержан г

In [6]:
#docs = pd.DataFrame(columns=['doc_id', 'text'])
#docs_id = [i for i in range(1, 28026+1)] #28026+1

In [6]:
from multiprocessing import Pool, Lock, Value
from time import sleep

mutex = Lock()
n_processed = Value('i', 0)

def func_wrapper(uid):
    res = clean_page(uid) 
    with mutex:
        # в этом блоке можно безопасно менять общие объекты для процессов
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

with Pool(processes=10) as pool:
    res = pool.map(func_wrapper, docs_id)

28020 objects are processed...

In [7]:
#docs['text'] = res
#docs['doc_id'] = docs_id

In [8]:
#docs.to_csv('processed_texts.csv', sep=';', index=False)

In [8]:
docs = pd.read_csv('processed_texts.csv', sep=';')

In [9]:
docs.head()

,doc_id,text
0,1,м б аншин центр репродукц генетик фертимед г м...
1,2,перевод кив кошельк перевод кив кошельк перево...
2,3,проект патрул времен реабилитац духовн существ...
3,4,блог клуб преподаван начальн класс порта профе...
4,5,быстр пониз холестерин высок холестерин симпто...


Группы обучения это train_groups.csv, группы предсказания test_groups.csv

doc_id - Уникальный идентификатор веб-страницы

group_id - Уникальный идентификатор группы веб-страниц, среди которой нужно найти подмножество одной тематики

pair_id - Уникальный идентификатор пары (group_id, doc_id)

target - Значение, которое нужно предсказать. 1 - страница настоящая, 0 - выброс

Важно отметить, обучающие и тестовые группы не пересекаются.

In [41]:
train_groups = pd.read_csv('train_groups.csv')
print (train_groups.shape)
train_groups.head()

(11690, 4)


,pair_id,group_id,doc_id,target
0,1,1,15731,0
1,2,1,14829,0
2,3,1,15764,0
3,4,1,17669,0
4,5,1,14852,0


In [42]:
test_groups = pd.read_csv('test_groups.csv')
print (test_groups.shape)
test_groups.head()

(16627, 3)


,pair_id,group_id,doc_id
0,11691,130,6710
1,11692,130,4030
2,11693,130,5561
3,11694,130,4055
4,11695,130,4247


Тестовое множество sample_submission.csv

In [43]:
sample_subm = pd.read_csv('sample_submission.csv')
print (sample_subm.shape)
sample_subm.head()

(16627, 2)


,pair_id,target
0,11691,0
1,11692,1
2,11693,0
3,11694,1
4,11695,0


In [44]:
tmp = pd.merge(docs, train_groups, how='inner', on='doc_id')
tmp.head()

,doc_id,text,pair_id,group_id,target
0,1,м б аншин центр репродукц генетик фертимед г м...,238,3,0
1,3,проект патрул времен реабилитац духовн существ...,2623,30,0
2,5,быстр пониз холестерин высок холестерин симпто...,4387,49,1
3,6,состав резюм энциклопед трудоустройств состав ...,4683,52,1
4,8,проект ладушк ладушк дет ран возраст дошкольни...,9712,108,0


In [45]:
import csv
tmp.drop(columns=['doc_id']).to_csv('for_fasttext.txt', 
                                 sep=' ', index=False, header=False, quoting=csv.QUOTE_NONE, quotechar="", escapechar=" ")

./fasttext skipgram -input for_fasttext.txt -output for_fasttext_vec.txt

In [52]:
tmp = tmp.fillna(value=' ')

In [53]:
ids = tmp[(tmp['group_id'] > 0) & (tmp['group_id'] < 131)]['doc_id'].values
targets = tmp[(tmp['group_id'] > 0) & (tmp['group_id'] < 131)]['target'].values
texts = tmp[(tmp['group_id'] > 0) & (tmp['group_id'] < 131)]['text'].values

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=10000)
X = vectorizer.fit_transform(texts)

In [55]:
X.shape

(11690, 10000)

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score
X_train, X_test, y_train, y_test = train_test_split(X, targets, test_size=0.2)
clf = LinearRegression()
clf.fit(X_train, y_train)

0.43458475540386804

In [59]:
y_pred = clf.predict(X_test)
y_pred = y_pred > 0.6
y_pred = y_pred.astype(int)
f1_score(y_test, y_pred)

0.4330254041570438

Основные правила:
1. Соревнование длится до конца курса
2. Баллы будут проставлены в зависимости от итоговой позици команды в рейтинге
3. При непродолении бейзлайна проект для команды не считается засчитанным
4. Команда может быть не более 4 человек
5. Весь код должен быть закомичен на гитхаб для проверки

Чего делать нельзя (карается незащитой проекта):
1. Обмениваться кодом между командами вне общего слака
2. Использовать ручную разметку

Чем можно пользоваться:
любыми алгоритмами, любыми дополнительными данными. 

Любые вопросы задавать в слаке @vikulin_seva

Перечислим здесь идеи, которые пришли нам самим в голову. Это только наше мнение, ему следовать не обязательно :)

1. Использовать нормализацию текста при подсчете текстовой похожести (см. лекцию 11 или интернет, библиотеку nltk, pymorphy)
2. Очевидно, просто число общих слов не идеальная метрика похожести. Как минимум, не учитывает длину, не учитывает, что бывают популярные слова (в, и, на), которые везде встречаются. 
3. Брать другую информацию из html страницы (url, body, meta...)
4. Попробовать лучше понять структуру в рамках одной группы с помощью методов кластеризации и добавить ее выход в признаки (см лекцию 8 или интернет)
5. Посмотреть в сторону методов детекции аномалий, которые явным образом в курсе не рассматриваются (см интернет, например, вот https://dyakonov.org/2017/04/19/поиск-аномалий-anomaly-detection/) и их выход добавлять в признаки
6. Попробовать разные алгоритмы машинного обучения

Интутивно кажется, что в этом конкурсе  важнее построение признаков, чем сам алгоритм, но, возможно, интуиция нас подводит.

Cоветы не только на этот конкурс: 
1. Смотреть на данные, только так можно придумать хорошее решение
2. Обычно, самые красивые решения являются самыми простыми (см http://alexanderdyakonov.narod.ru/intro2datamining.pdf http://alexanderdyakonov.narod.ru/lpotdyakonov.pdf). Постарайтесь их найти :)
3. Не доверять полностью публичному значению качества на kaggle, ВСЕГДА использовать валидацию
4. Пробовать усреднять предсказания разных моделей, это может работать лучше, чем каждая по отдельности. 


Удачи!